<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/165_Agent_Examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1) Job Application Agent

This is a classic **conditional routing** assignment in LangGraph. You already have the skeleton. Let’s fill it in step by step:

---

## 📝 What you need to do

1. **Implement `categorize_candidate`** so it checks years of experience.
2. **Add graph nodes** for each step (`categorize_candidate`, `schedule_interview`, `assign_skills_test`).
3. **Add conditional edges** so the workflow routes to the right node depending on the candidate.

---

## 🏃 What Happens

* Alice has 6 years → categorized as `"senior"` → routed to `schedule_interview`.
* Bob has 3 years → categorized as `"junior"` → routed to `assign_skills_test`.


In [ ]:
from typing import TypedDict
from langgraph.graph import END, START, StateGraph

# Define the structure of the input state (job application)
class JobApplication(TypedDict):
    applicant_name: str
    years_experience: int
    status: str

# Function to categorize candidates
def categorize_candidate(application: JobApplication):
    if application["years_experience"] >= 5:
        return {"status": "senior"}
    else:
        return {"status": "junior"}

# Function for interview scheduling
def schedule_interview(application: JobApplication):
    print(f"Candidate {application['applicant_name']} is shortlisted for an interview.")
    return {"status": "Interview Scheduled"}

# Function for skills test
def assign_skills_test(application: JobApplication):
    print(f"Candidate {application['applicant_name']} is assigned a skills test.")
    return {"status": "Skills Test Assigned"}

# Create the state graph
graph = StateGraph(JobApplication)

# Add nodes
graph.add_node("categorize", categorize_candidate)
graph.add_node("interview", schedule_interview)
graph.add_node("skills_test", assign_skills_test)

# Entry point
graph.set_entry_point("categorize")

# Conditional edges
graph.add_conditional_edges(
    "categorize",
    lambda state: state["status"],  # decide based on status
    {
        "senior": "interview",
        "junior": "skills_test"
    }
)

# End edges
graph.add_edge("interview", END)
graph.add_edge("skills_test", END)

# Compile the workflow
runnable = graph.compile()

# Simulate job applications
print(runnable.invoke({"applicant_name": "Alice", "years_experience": 6, "status": ""}))
print(runnable.invoke({"applicant_name": "Bob", "years_experience": 3, "status": ""}))



### 1. What does `add_conditional_edges` do?

Normally in a graph you connect Node A → Node B. But sometimes, you want a **decision point**:

* If condition = X → go to Node B
* If condition = Y → go to Node C

That’s what `add_conditional_edges` does: it lets you route the workflow *dynamically* based on the state.

---

### 2. The role of `lambda state: state["status"]`

* Every node in LangGraph takes the current **state** (a dictionary that matches your `TypedDict`) as input.
* A **lambda** is just a small inline function.
* `lambda state: state["status"]` means:

  > “Look at the current state and return the value of the `status` field.”

So:

* If the state has `{"status": "senior"}` → it returns `"senior"`.
* If the state has `{"status": "junior"}` → it returns `"junior"`.

---

### 3. How the mapping works

You then tell LangGraph how to map that return value to nodes:

```python
{
    "senior": "interview",
    "junior": "skills_test"
}
```

So:

* If `lambda state` returns `"senior"`, LangGraph routes to node `"interview"`.
* If it returns `"junior"`, LangGraph routes to `"skills_test"`.

---

### 4. Why a lambda?

Because it gives you **flexibility**. You could have written something fancier, e.g.:

```python
lambda state: "senior" if state["years_experience"] >= 5 else "junior"
```

This way you don’t even need to store `"status"` in state — you compute it on the fly.

---

✅ **Summary**:
The `lambda state: state["status"]` is just a tiny function that looks at the workflow’s current state and tells the graph which edge to follow. Think of it as the “decision rule” at a fork in the workflow.



## 2) Apply Reducer

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage
from operator import add
from langgraph.graph import END, START, StateGraph
from langchain_core.messages import AIMessage, HumanMessage


# Define chatbot state with accumulated orders
class OrderState(TypedDict):
    messages: list[AnyMessage]
    order_id: int


# Step 1: Take the food order
def take_order(state: OrderState):
    return {"messages": [AIMessage(content="Processing your order?")]}


# Step 2: Confirm the order
def confirm_order(state: OrderState):
    return {"messages": [AIMessage(content="Your order has been placed!")], "order_id": 1}


# Build chatbot conversation flow
graph_builder = StateGraph(OrderState)

# Add nodes
graph_builder.add_node("take_order", take_order)
graph_builder.add_node("confirm_order", confirm_order)

# Define conversation flow
graph_builder.add_edge(START, "take_order")
graph_builder.add_edge("take_order", "confirm_order")
graph_builder.add_edge("confirm_order", END)

# Compile chatbot
chatbot = graph_builder.compile()

# Simulate a conversation
test_input = "I want a burger."

messages = chatbot.invoke({"messages": [HumanMessage(content=test_input)]})

for message in messages["messages"]:
    print(f"Message: {message.content}")

print("Total Orders: ", messages["order_id"])


Right now your `OrderState` looks like this:

```python
class OrderState(TypedDict):
    messages: list[AnyMessage]
    order_id: int
```

This means **by default** LangGraph will use *last-write-wins*: if multiple nodes update `messages` or `order_id`, the later one just overwrites the previous value.

But your assignment is to **accumulate** both `messages` and `order_id` using reducers. That’s where `Annotated` + `operator.add` come in.

---

## ✅ How to Apply Reducers

Here’s the change:

```python
from typing import Annotated, TypedDict
from operator import add
from langchain_core.messages import AnyMessage

# Define chatbot state with reducers
class OrderState(TypedDict):
    # Accumulate all messages instead of overwriting
    messages: Annotated[list[AnyMessage], add]
    # Add order IDs together instead of overwriting
    order_id: Annotated[int, add]
```

---

### 🔎 What this does:

* `messages: Annotated[list[AnyMessage], add]`
  → If one node returns `{"messages": [AIMessage("hi")]}` and another node later returns `{"messages": [AIMessage("bye")]}`, LangGraph will combine them into `[AIMessage("hi"), AIMessage("bye")]` instead of overwriting.

* `order_id: Annotated[int, add]`
  → If one node sets `{"order_id": 1}` and another later sets `{"order_id": 1}`, the reducer will add them up → final value `2`.

This way, each new order increments the total count, instead of resetting.

---

## ✨ Updated Section in Your Code

```python
# Define chatbot state with accumulated orders
class OrderState(TypedDict):
    messages: Annotated[list[AnyMessage], add]
    order_id: Annotated[int, add]
```

The rest of your workflow (`take_order`, `confirm_order`, etc.) stays the same.




In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage
from operator import add
from langgraph.graph import END, START, StateGraph
from langchain_core.messages import AIMessage, HumanMessage


# Define chatbot state with accumulated orders
# class OrderState(TypedDict):
#     messages: list[AnyMessage]
#     order_id: int

# Define chatbot state with reducers
class OrderState(TypedDict):
    # Accumulate all messages instead of overwriting
    messages: Annotated[list[AnyMessage], add]
    # Add order IDs together instead of overwriting
    order_id: Annotated[int, add]

# Step 1: Take the food order
def take_order(state: OrderState):
    return {"messages": [AIMessage(content="Processing your order?")]}


# Step 2: Confirm the order
def confirm_order(state: OrderState):
    return {"messages": [AIMessage(content="Your order has been placed!")], "order_id": 1}


# Build chatbot conversation flow
graph_builder = StateGraph(OrderState)

# Add nodes
graph_builder.add_node("take_order", take_order)
graph_builder.add_node("confirm_order", confirm_order)

# Define conversation flow
graph_builder.add_edge(START, "take_order")
graph_builder.add_edge("take_order", "confirm_order")
graph_builder.add_edge("confirm_order", END)

# Compile chatbot
chatbot = graph_builder.compile()

# Simulate a conversation
test_input = "I want a burger."

messages = chatbot.invoke({"messages": [HumanMessage(content=test_input)]})

for message in messages["messages"]:
    print(f"Message: {message.content}")

print("Total Orders: ", messages["order_id"])


👍 Next — you want to move away from writing a custom `TypedDict` (like `OrderState`) and instead just use LangGraph’s built-in **`MessagesState`**.

---

### 🔎 What `MessagesState` is

LangGraph provides `MessagesState` as a convenience type when your main state is a list of messages (`HumanMessage`, `AIMessage`, etc.).
It already has the reducer `Annotated[List[AnyMessage], add]` baked in, so messages automatically accumulate across nodes.

---

### ✅ How to refactor your workflow

Right now you have:

```python
from typing import Annotated, TypedDict
from operator import add
from langchain_core.messages import AnyMessage

class OrderState(TypedDict):
    messages: Annotated[list[AnyMessage], add]
    order_id: Annotated[int, add]
```

You can replace that with:

```python
from langgraph.graph import MessagesState
from typing import Annotated
from operator import add

# Extend MessagesState to add order_id with reducer
class OrderState(MessagesState):
    order_id: Annotated[int, add]
```

---

### ✨ Benefits

* You don’t need to redefine `messages` — `MessagesState` already handles it.
* You still keep `order_id` as an accumulator by extending it.
* Cleaner and more idiomatic LangGraph code.

---

So effectively:

* ✅ Use `MessagesState` for conversation memory.
* ✅ Add extra fields (like `order_id`) on top.



In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage
from operator import add
from langgraph.graph import END, START, StateGraph
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import MessagesState

# Extend MessagesState to add order_id with reducer
class OrderState(MessagesState):
    order_id: Annotated[int, add]

# Step 1: Take the food order
def take_order(state: OrderState):
    return {"messages": [AIMessage(content="Processing your order?")]}


# Step 2: Confirm the order
def confirm_order(state: OrderState):
    return {"messages": [AIMessage(content="Your order has been placed!")], "order_id": 1}


# Build chatbot conversation flow
graph_builder = StateGraph(OrderState)

# Add nodes
graph_builder.add_node("take_order", take_order)
graph_builder.add_node("confirm_order", confirm_order)

# Define conversation flow
graph_builder.add_edge(START, "take_order")
graph_builder.add_edge("take_order", "confirm_order")
graph_builder.add_edge("confirm_order", END)

# Compile chatbot
chatbot = graph_builder.compile()

# Simulate a conversation
test_input = "I want a burger."

messages = chatbot.invoke({"messages": [HumanMessage(content=test_input)]})

for message in messages["messages"]:
    print(f"Message: {message.content}")

print("Total Orders: ", messages["order_id"])




Because `MessagesState` is just:

```python
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add]
```

with the **`add` reducer baked in**, every node that appends to `state["messages"]` causes the list to grow.

---

### 🔎 What this means in practice

* **Yes**: at any point in execution you can inspect `state["messages"]` and see **the full conversation so far** (all `HumanMessage`, `AIMessage`, `SystemMessage`, etc.).
* It’s not overwriting — it’s **accumulating**.
* Think of it as a conversation log that keeps expanding as the graph progresses.

Example:

```python
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, AIMessage

state = MessagesState(messages=[])

# node 1 appends a human message
state["messages"].append(HumanMessage(content="Hello"))

# node 2 appends an AI response
state["messages"].append(AIMessage(content="Hi there!"))

print([m.content for m in state["messages"]])
# ["Hello", "Hi there!"]
```

At this point, **any downstream node sees the *entire* conversation**.

---

### ⚖️ Why this matters

* Agents that need **conversation history** (like a chatbot) always get the context.
* You don’t need to manage reducers or manually merge lists — it’s automatic.
* You can always branch, filter, or truncate `messages` if you want to manage context length.






### 🔹 When `MessagesState` *is* the best option

Use `MessagesState` if:

* The **main unit of state is the conversation** (chat turns).
* You want to **persist and accumulate messages** between nodes without redefining reducers every time.
* Your workflow is essentially “agents talking” (customer support bot, orchestrator chat loop, multi-turn assistant).

It’s a convenience wrapper:

```python
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add]
```

So if your workflow is **mostly messages + a few extras**, you can just subclass it and add extra fields:

```python
class OrderState(MessagesState):
    order_id: Annotated[int, add]
```

---

### 🔹 When a custom `TypedDict` is better

Stick to your own `TypedDict` if:

* You don’t really care about conversation logs.
* The workflow state is **structured data**, like `order_id`, `cart_items`, `payment_status`, etc.
* Messages are incidental, not the core.

Example: an e-commerce order processor doesn’t necessarily need `MessagesState`. It might make more sense to define:

```python
class OrderState(TypedDict):
    order_id: Annotated[int, add]
    items: Annotated[list[str], add]
    status: str
```

---

### ⚖️ In your case (order processing workflow):

* If the focus is **simulating an agent talking with a customer about their order**, then yes ✅ `MessagesState` is a neat fit.
* If the focus is more like a **backend pipeline (orders flowing through steps)**, then a plain `TypedDict` with reducers is the better, cleaner choice.




## 3) Doctor Visit Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver

# -------------------------
# Tools
# -------------------------
@tool
def check_symptoms(symptom: str):
    """Provides possible conditions based on the symptom described."""
    conditions = {
        "fever": ["Flu", "COVID-19", "Common Cold"],
        "cough": ["Bronchitis", "Pneumonia", "Common Cold"],
        "headache": ["Migraine", "Tension Headache", "Sinus Infection"],
    }
    return conditions.get(symptom.lower(), ["No specific conditions found. Please consult a doctor."])

@tool
def book_doctor_appointment(specialty: str, date: str, time: str):
    """Books an appointment with a doctor based on the required specialty."""
    available_specialties = ["General Physician", "Cardiologist", "Neurologist", "Pediatrician"]
    if specialty in available_specialties:
        return f"Appointment booked with {specialty} on {date} at {time}."
    else:
        return f"Sorry, no available {specialty} at this time."

tools = [check_symptoms, book_doctor_appointment]

# -------------------------
# LLM
# -------------------------
llm = ChatOpenAI(model="gpt-4o-mini")  # you can change to gpt-3.5 or gpt-4
llm_with_tools = llm.bind_tools(tools)

# -------------------------
# Nodes
# -------------------------
# ToolNode: executes actual tool calls
tool_node = ToolNode(tools)

# Model call node
def call_model(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# Routing: if AIMessage contains tool calls, go to ToolNode
def should_continue(state: MessagesState):
    last_msg = state["messages"][-1]
    if isinstance(last_msg, AIMessage) and last_msg.tool_calls:
        return "tools"
    return END

# -------------------------
# Workflow
# -------------------------
workflow = StateGraph(MessagesState)

workflow.add_node("model", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "model")
workflow.add_conditional_edges("model", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "model")

# -------------------------
# Compile & Run
# -------------------------
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}

# Step 1: Check Symptoms
response = graph.invoke(
    {"messages": [HumanMessage(content="I have a fever. Can you tell me what this condition might be?")]},
    config
)

print(response["messages"][-1])
conditions = response["messages"][-1].content
print("\n🔍 **Possible Conditions Based on Symptoms:**")
print(conditions)

# Step 2: Book Doctor Appointment
response = graph.invoke(
    {"messages": [HumanMessage(content="Book an appointment for these conditions with a General Physician for tomorrow at 10 AM.")]},
    config
)

final_response = response["messages"][-1].content
print("\n📅 **Doctor Appointment Confirmation:**")
print(final_response)


👌 — this is one of those design subtleties in LangGraph that’s important to get clear.

---

### 🔹 ToolNode vs add_node(tool)

```python
tools = [check_symptoms, book_doctor_appointment]
tool_node = ToolNode(tools)
```

instead of directly:

```python
tool_node = ToolNode([check_symptoms, book_doctor_appointment])
```

That’s honestly just **style/clarity**. Both are valid.
I made a `tools` list so it could be reused in two places:

* ✅ `llm_with_tools = llm.bind_tools(tools)`
* ✅ `ToolNode(tools)`

This way you don’t duplicate the list of tools and risk mismatches. But if you don’t need reuse, you can inline them directly.

So **it’s not because of binding** — binding and ToolNode are two separate things:

* `llm.bind_tools(tools)` → tells the LLM how to *call* tools (generates structured tool calls).
* `ToolNode(tools)` → actually *executes* the tool calls when they appear in the messages.

You need both if you want a ReAct-style loop (model decides to call → ToolNode executes → model continues).

---

### 🔹 Difference: ToolNode vs adding tools as normal nodes

You *could* just do:

```python
workflow.add_node("check", check_symptoms)
workflow.add_node("book", book_doctor_appointment)
```

But that would mean:

* You must manually decide when to call which tool.
* The graph edges would hard-code “user → check_symptoms” or “user → book_doctor_appointment.”
* No dynamic “AI decides tool usage.”

Whereas with `ToolNode`:

* It’s a **generic execution node**: it looks at the last `AIMessage` for tool calls, then routes them to the correct function automatically.
* You don’t have to manually wire edges for every tool. Add 10 tools? Still one `ToolNode`.
* Works naturally with `bind_tools`, since the LLM emits structured tool calls and ToolNode just executes them.

---

### ⚖️ So in short

* **ToolNode** = dynamic, reusable executor for many tools. It plugs into the “model → decide → tool → model” loop.
* **Plain add_node(tool_fn)** = static function call. Use if you want fixed steps in the workflow (like `summarizer → writer → editor`).






Let’s make the contrast super clear:

---

### 🔹 With `ToolNode(tools)`

```python
workflow.add_node("model", call_model)
workflow.add_node("tools", tool_node)
```

* `call_model` = LLM generates output, which *might* include a tool call.
* If the output contains a tool call → we route into `tool_node`.
* `ToolNode` looks at the tool call metadata (`AIMessage.tool_calls`) and dispatches to the correct tool (`check_symptoms` or `book_doctor_appointment`).
* Then the result goes back into the model, which can decide to call another tool, or just finish.

👉 This is **dynamic** — the LLM decides which tool(s) to call, in what order, and how many times.

---

### 🔹 With plain `add_node(tool1)` + `add_node(tool2)`

```python
workflow.add_node("check", check_symptoms)
workflow.add_node("book", book_doctor_appointment)
```

* Each node is just a Python function.
* You must wire the edges explicitly:

  ```python
  workflow.add_edge("check", "book")
  ```
* That means `check → book` is always executed in sequence, regardless of whether the user wanted it.
* No “choice” — it’s a fixed pipeline.

👉 This is **static** — you’re building a deterministic workflow where tools always run in a pre-set order.

---

### ⚖️ When to use which

* **Use `ToolNode(tools)`** when you want the LLM to behave like an *agent*, dynamically choosing tools and chaining them.
* **Use plain `add_node`** when you want a *workflow pipeline* (e.g., summarizer → writer → editor) where each step always happens.

---

So your example:

```python
workflow.add_node("model", call_model)
workflow.add_node("tools", tool_node)
```

is the canonical **agent pattern** in LangGraph:
`START → model → (maybe tools) → model → END`.




## RAG

In [ ]:
from typing import List, TypedDict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# Medical Knowledge Sources (Example URLs)
medical_info_urls = [
    "https://www.who.int/health-topics",
    "https://www.mayoclinic.org/diseases-conditions",
    "https://medlineplus.gov/symptoms.html",
    "https://www.webmd.com/a-to-z-guides/diseases-conditions",
]

# Load Medical Data
docs = [WebBaseLoader(url).load() for url in medical_info_urls]
docs_list = [item for sublist in docs for item in sublist]

# Split Medical Information
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

# Store and Retrieve Medical Data with ChromaDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="medical-diagnosis",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

# Prompt for Medical Diagnosis
prompt = ChatPromptTemplate.from_template(
    """
    You are an AI doctor providing preliminary medical diagnoses based on symptoms.
    Use ONLY the retrieved medical information in {context} to ground your analysis.
    If information is insufficient, say so and suggest seeing a clinician.

    Patient Symptoms: {question}

    Relevant Medical Information:
    {context}

    AI-Powered Diagnosis:
    """
)
model = ChatOpenAI()
diagnosis_chain = prompt | model | StrOutputParser()

# Define Graph State
class MedicalDiagnosisGraphState(TypedDict):
    question: str
    retrieved_data: List[str]
    diagnosis: str

# --- Implementations ---

def retrieve_medical_data(state: MedicalDiagnosisGraphState):
    """Use the vector retriever to gather relevant medical passages for the question."""
    query = state["question"]
    docs = retriever.get_relevant_documents(query)
    # keep just the text; cap to a handful to control prompt size
    passages = [d.page_content for d in docs][:8]
    return {"retrieved_data": passages}

def analyze_medical_diagnosis(state: MedicalDiagnosisGraphState):
    """Run the RAG prompt: combine retrieved context + question -> diagnosis."""
    context = "\n\n---\n\n".join(state["retrieved_data"]) if state["retrieved_data"] else "No context retrieved."
    diagnosis_text = diagnosis_chain.invoke({"question": state["question"], "context": context})
    return {"diagnosis": diagnosis_text}

def create_medical_diagnosis_workflow():
    """Build a simple two-node LangGraph: retrieve -> analyze -> END."""
    builder = StateGraph(MedicalDiagnosisGraphState)
    builder.add_node("retrieve", retrieve_medical_data)
    builder.add_node("analyze", analyze_medical_diagnosis)

    builder.set_entry_point("retrieve")
    builder.add_edge("retrieve", "analyze")
    builder.add_edge("analyze", END)

    checkpointer = MemorySaver()
    return builder.compile(checkpointer=checkpointer)

# Execute the Workflow
medical_diagnosis_graph = create_medical_diagnosis_workflow()

inputs = {"question": "I have a persistent cough, fever, and shortness of breath. What could it be?"}

response = medical_diagnosis_graph.invoke(inputs)

print("\n--- AI MEDICAL DIAGNOSIS ---")
print(response["diagnosis"])


## Human in the Loop

In [ ]:
from typing import List, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.types import interrupt
from langgraph.types import Command


class CodingAssistantState(TypedDict):
    task: str
    code: str
    tests: str


model = ChatOpenAI()

code_prompt = ChatPromptTemplate.from_template("Generate Python code for: {task}")
test_prompt = ChatPromptTemplate.from_template("Write unit tests for this code:\n{code}")

code_chain = code_prompt | model | StrOutputParser()
test_chain = test_prompt | model | StrOutputParser()


def generate_code(state):
    print("Generate Code")
    code = code_chain.invoke({"task": state["task"]})
    return Command(goto="human_review", update={"code": code})


# TODO
def human_review(state):
    value = interrupt({
        "question": "Are you ok with the code.Type yes or no"
    })
    if value == "yes":
        return Command(goto="create_tests")
    else:
        return Command(goto=END)


def create_tests(state):
    tests = test_chain.invoke({"code": state["code"]})
    return Command(goto=END, update={"tests": tests})


def create_coding_assistant_workflow():
    workflow = StateGraph(CodingAssistantState)
    workflow.add_node("generate_code", generate_code)
    workflow.add_node("human_review", human_review)
    workflow.add_node("create_tests", create_tests)
    workflow.set_entry_point("generate_code")
    return workflow.compile(checkpointer=MemorySaver())


# Run the Workflow
coding_assistant = create_coding_assistant_workflow()
inputs = {"task": "Create a function to reverse a string in Python."}
thread = {"configurable": {"thread_id": 1}}
result = coding_assistant.invoke(inputs, config=thread)
# TODO: Handle Interrupt
print("\n-----Generated Code--------")
print(result["code"])
tasks = coding_assistant.get_state(config=thread).tasks
print(tasks)
task = tasks[0]
question = task.interrupts[0].value.get("question")
user_input = input(question)
result = coding_assistant.invoke(Command(resume=user_input),config=thread)

print("\n--- Generated Tests ---")
print(result.get("tests", "No code or tests generated"))


You’ve hit on one of the **biggest conceptual differences** between two styles in LangChain/LangGraph:

---

## 🔹 1. Chains (`prompt | model | parser`)

This is what you see in your **human-in-the-loop** script:

```python
code_prompt = ChatPromptTemplate.from_template("Generate Python code for: {task}")
code_chain = code_prompt | model | StrOutputParser()
```

That `|` operator is just syntactic sugar for **piping one component’s output into the next**:

* Prompt template fills in text (`{task}`).
* Model runs on that text.
* Output parser converts the raw model response into a Python type (here, `str`).

👉 Think of a `chain` as a **mini data pipeline** that executes *within a single node*. It’s sequential but self-contained. Great for small, linear transformations.

---

## 🔹 2. Graph Nodes (sequential state machine)

In your earlier assignments, we did this:

```python
builder.add_node("summarizer", summarize)
builder.add_node("writer", write_report)
builder.add_edge("summarizer", "writer")
```

Here, the **graph engine** controls flow. Each node is a function that takes state and returns state (or a `Command` with routing). This lets you branch, loop, add parallel paths, or insert human review.

👉 Think of a `graph` as the **workflow orchestrator**.

---

## 🔹 Key Difference

* **Chains** are *intra-node pipelines*: "prompt → LLM → parser".

  * Lightweight, sequential, reusable.
  * Ideal inside a node that should just “do one thing.”
* **Graphs** are *inter-node workflows*: "node A → node B → …".

  * Heavy-duty orchestration.
  * Ideal when you want branching, retries, human feedback, or multi-step agent reasoning.

---

## 🔹 Why your human-in-the-loop script uses both

* **Chains** (`code_chain`, `test_chain`) encapsulate *atomic reasoning tasks*: generate code, generate tests.
* **Graph nodes** wire them together and insert interruptions for **human review** before continuing.

So:

* Chain = **how a node thinks**.
* Graph = **how nodes talk**.

---

✅ The Human-in-the-Loop design is possible because `interrupt()` pauses the graph and waits for user input, something you *can’t* do with just a linear chain.





### 🔹 1. Where LangChain stops and LangGraph begins

* **LangChain** gives you building blocks: `PromptTemplate`, `LLM`, `OutputParser`, and the `|` operator to compose them into **chains**.
* **LangGraph** is the orchestrator: it runs chains as nodes, controls flow, and adds features like `interrupt()` for human review, retries, error edges, and parallelism.

So your code generator agent is really:

* Node logic = LangChain chain (`prompt | model | parser`)
* Workflow wiring = LangGraph

---

### 🔹 2. Human in the loop is special

Most graphs run end-to-end automatically. But with `interrupt("review_code")` you’re saying:

* “Pause here, wait for human input.”
* The graph saves state, and when you provide feedback (like ✅ accept or ✏️ edit), it resumes right where it left off.

That’s super powerful for safety-critical domains (coding, medicine, law) where you don’t want unchecked LLM outputs.

---

### 🔹 3. State is still the backbone

Even in this setup, the `state` carries:

* Conversation messages (if you subclass `MessagesState`)
* Intermediate artifacts like `generated_code`, `tests`, `feedback`
* Control variables (like a counter if you want multiple iterations)

Because the graph persists state, you can resume after a crash or even hand off the workflow between people.

---

### 🔹 4. You can mix patterns

This agent is already combining **Chains** (intra-node pipelines) with **Graph orchestration** (inter-node workflow). But you could layer on:

* **Reflection** → after human approval, add a loop for the model to self-critique and improve code.
* **Parallelism** → generate multiple code candidates in parallel and vote.
* **Routing** → detect if a request is about Python, JS, or SQL and send to the right code-gen node.

---

### 🔹 5. Costs & performance

Every pause + human review = a potential delay in throughput. But the trade-off is higher quality and safer outputs. Usually worth it when mistakes are expensive.

---

✅ So the big picture: this agent is a **safe code-gen pipeline** where LangChain handles the micro-steps (prompts → LLM → parser) and LangGraph handles the macro orchestration (generate → pause for review → test → final output).




Great catch 🙌 — you noticed that in your **code generator agent** the workflow is wrapped inside a function:

```python
def create_coding_assistant_workflow():
    workflow = StateGraph(CodingAssistantState)
    ...
    return workflow.compile(checkpointer=MemorySaver())
```

Whereas before we were just building the graph at the top level.

---

### 🔹 Why wrap it in a definition?

1. **Reusability**

   * By putting it in a function, you can call `create_coding_assistant_workflow()` as many times as you like.
   * Each call gives you a *fresh compiled workflow* with its own state and memory.
   * Super useful for tests, multiple agents, or multiple sessions.

2. **Encapsulation**

   * Keeps all the graph-building logic in one place.
   * Makes your main script cleaner (e.g., `workflow = create_coding_assistant_workflow()` instead of 15 lines of builder code).

3. **Parameterization**

   * You can add arguments to the function later (`use_persistent_memory=True`, `model="gpt-4o-mini"`, etc.) to generate slightly different workflows with minimal code duplication.

4. **Checkpointer lifecycle**

   * Notice how it returns `workflow.compile(checkpointer=MemorySaver())`.
   * This ensures that *every* workflow instance you spin up has its own checkpointer configured.
   * Without wrapping, you’d risk reusing global memory unintentionally.

---

### 🔹 Difference from your earlier sequential-node graphs

Earlier, we just did something like:

```python
workflow = StateGraph(MyState)
workflow.add_node("summarizer", summarize)
workflow.set_entry_point("summarizer")
app = workflow.compile()
```

That’s fine for a single static graph in a notebook.
But when you want to package it into a **reusable component** (like an agent or service), the function-based builder pattern is cleaner.

---

✅ So: **same graph mechanics, different packaging style**.

* Top-level: good for quick demos.
* Function factory: best for reusable, parameterized agents.

